<a href="https://colab.research.google.com/github/aayushhh-operator/Crime-Forecasting/blob/daksh/LA_Crime_finall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install holidays

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.7/824.7 kB 13.4 MB/s eta 0:00:00


In [2]:
!pip install prophet==1.1.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 5.9 MB/s eta 0:00:00


In [3]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 14.9 MB/s eta 0:00:00


In [4]:
!pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 5.6 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import folium
from folium.plugins import HeatMap
import plotly.express as px

In [6]:
df = pd.read_csv('/content/Clean_LA_Crime.csv')

In [7]:
df['DateTime'] = pd.to_datetime(df['DATE OCC'].astype(str) + ' ' + df['TIME OCC'].astype(str))

# Combine 24 Hours

In [8]:
# Load and preprocess your data (assuming it's already loaded and 'df' is your DataFrame)
df['DateTime'] = pd.to_datetime(df['DATE OCC'].astype(str) + ' ' + df['TIME OCC'].astype(str))
df_resampled = df.resample('H', on='DateTime').size()

# Split data into training and testing
train_data = df_resampled[:-24]
test_data = df_resampled[-24:]

# ARIMA model
arima_model = ARIMA(train_data, order=(5, 1, 0))  # Adjust order based on ACF/PACF plots or grid search
arima_model_fit = arima_model.fit()
arima_forecast = arima_model_fit.forecast(steps=24)

# Prepare ARIMA forecasts as regressor for Prophet
arima_forecast_df = pd.DataFrame({
    'ds': pd.date_range(start=train_data.index[-1], periods=25, freq='H')[1:],
    'arima_forecast': arima_forecast
})

# Prophet model using ARIMA forecast as an additional regressor
df_prophet = train_data.reset_index()
df_prophet.columns = ['ds', 'y']

prophet_model = Prophet()
prophet_model.add_regressor('arima_forecast')

# Merging ARIMA forecasts with training data for Prophet
prophet_train = df_prophet.merge(arima_forecast_df, on='ds', how='outer')

# Ensure 'y' is filled for the ARIMA forecast periods before dropping NA
prophet_train['y'] = prophet_train['y'].fillna(method='ffill') # Forward fill missing 'y' values

# Now drop rows with any remaining missing values (likely in 'arima_forecast' for early periods)
prophet_train = prophet_train.dropna()

# Check if you still have enough data
if len(prophet_train) < 2:
    raise ValueError("Not enough data after merging and filling to fit Prophet. You may need more data or adjust your forecast periods.")

prophet_model.fit(prophet_train)  # Fit on non-na to avoid errors from initial missing ARIMA forecasts

# Create future dataframe for Prophet predictions and merge ARIMA forecasts
future = prophet_model.make_future_dataframe(periods=24, freq='H')

# Fill NaN values in 'arima_forecast' with 0
# to prevent issues with Prophet requiring all regressors
future['arima_forecast'] = future.merge(arima_forecast_df, on='ds', how='left')['arima_forecast'].fillna(0)


# Predict with Prophet using ARIMA forecast
forecast_prophet = prophet_model.predict(future)

# Calculate MSE and RMSE
from sklearn.metrics import mean_squared_error

mse_prophet = mean_squared_error(test_data.values, forecast_prophet['yhat'][-24:])
rmse_prophet = np.sqrt(mse_prophet)
print({'MSE': mse_prophet, 'RMSE': rmse_prophet})

<ipython-input-8-e4542ee7b6b5>:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('H', on='DateTime').size()
<ipython-input-8-e4542ee7b6b5>:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'ds': pd.date_range(start=train_data.index[-1], periods=25, freq='H')[1:],
<ipython-input-8-e4542ee7b6b5>:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  prophet_train['y'] = prophet_train['y'].fillna(method='ffill') # Forward fill missing 'y' values
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints gre

{'MSE': 3.0, 'RMSE': 1.7320508075688772}


/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1854: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(


# Combine 1 Week

In [9]:
# Splitting data into training and testing sets for one week forecast
train_data = df_resampled[:-168]
test_data = df_resampled[-168:]

# Fit ARIMA model and forecast for 168 hours
arima_model = ARIMA(train_data, order=(5, 1, 0))
arima_model_fit = arima_model.fit()
arima_forecast = arima_model_fit.forecast(steps=168)

# Prepare the data for Prophet with ARIMA forecasts
train_data_reset = train_data.reset_index()
train_data_reset.columns = ['ds', 'y']
arima_forecast_df = pd.DataFrame({
    'ds': pd.date_range(start=train_data.index[-1], periods=169, freq='H')[1:],
    'arima_forecast': arima_forecast
})

# Fit Prophet model using ARIMA forecasts as an additional regressor
prophet_model = Prophet()
prophet_model.add_regressor('arima_forecast')
# Instead of an outer merge, use a left merge to keep all training data
prophet_train = pd.merge(train_data_reset, arima_forecast_df, on='ds', how='left')

# Fill NaN values in 'arima_forecast' with 0 before fitting
# to prevent issues with Prophet requiring all regressors to have values
prophet_train['arima_forecast'] = prophet_train['arima_forecast'].fillna(0)

prophet_model.fit(prophet_train)  # Fit on the training data with filled NaNs
# --- CHANGES END HERE ---

# Create future dataframe for Prophet predictions
future = prophet_model.make_future_dataframe(periods=168, freq='H')

# Instead of merging, directly fill 'arima_forecast' in 'future' using values from 'arima_forecast_df'
future['arima_forecast'] = future['ds'].map(arima_forecast_df.set_index('ds')['arima_forecast']).fillna(0)

# Predict with Prophet
forecast_prophet = prophet_model.predict(future)

# Calculate MSE and RMSE
mse_prophet = mean_squared_error(test_data.values, forecast_prophet['yhat'][-168:])
rmse_prophet = np.sqrt(mse_prophet)

# Output model metrics
model_metrics = {'MSE': [mse_prophet], 'RMSE': [rmse_prophet]}
print(model_metrics)


<ipython-input-9-d99665fc1434>:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'ds': pd.date_range(start=train_data.index[-1], periods=169, freq='H')[1:],
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdgkgsmk6/umlet96g.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdgkgsmk6/z3n8vkco.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=13650', 'data', 'file=/tmp/tmpdgkgsmk6/umlet96g.json', 'init=/tmp/tmpdgkgsmk6/z3n8vkco.json', 'output', 'file=/tmp/tmpdgkgsmk6/prophet_modelj7jk1fhg/prophet_model-20250316201313.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
20:13:13 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
20:13:37 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-

{'MSE': [62.59408538858535], 'RMSE': [7.911642395140553]}


# Combine 1 Month

In [10]:
# Splitting data into training and testing sets for one month forecast
train_data = df_resampled[:-720]
test_data = df_resampled[-720:]

# Fit ARIMA model and forecast for 720 hours
arima_model = ARIMA(train_data, order=(5, 1, 0))  # Adjust ARIMA order as necessary
arima_model_fit = arima_model.fit()
arima_forecast = arima_model_fit.forecast(steps=720)

# Prepare the data for Prophet with ARIMA forecasts
train_data_reset = train_data.reset_index()
train_data_reset.columns = ['ds', 'y']
arima_forecast_df = pd.DataFrame({
    'ds': pd.date_range(start=train_data.index[-1], periods=721, freq='H')[1:],  # Generate dates for ARIMA forecasts
    'arima_forecast': arima_forecast
})

# Fit Prophet model using ARIMA forecasts as an additional regressor
prophet_model = Prophet()
prophet_model.add_regressor('arima_forecast')

# Use a left merge to keep all training data and fill missing ARIMA forecasts with 0
prophet_train = pd.merge(train_data_reset, arima_forecast_df, on='ds', how='left')
prophet_train['arima_forecast'] = prophet_train['arima_forecast'].fillna(0) # Fill missing ARIMA forecasts

prophet_model.fit(prophet_train)  # Fit on the training data

# Create future dataframe for Prophet predictions
future = prophet_model.make_future_dataframe(periods=720, freq='H')

# Fill 'arima_forecast' in 'future' using values from 'arima_forecast_df', fill missing with 0
future['arima_forecast'] = future['ds'].map(arima_forecast_df.set_index('ds')['arima_forecast']).fillna(0)

# Predict with Prophet
forecast_prophet = prophet_model.predict(future)

# Calculate MSE and RMSE
mse_prophet = mean_squared_error(test_data, forecast_prophet['yhat'][-720:])
rmse_prophet = np.sqrt(mse_prophet)

# Output model metrics
model_metrics = {'MSE': [mse_prophet], 'RMSE': [rmse_prophet]}
print(model_metrics)


<ipython-input-10-5e78687e8280>:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'ds': pd.date_range(start=train_data.index[-1], periods=721, freq='H')[1:],  # Generate dates for ARIMA forecasts
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdgkgsmk6/uvb58qnc.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdgkgsmk6/g24ljnu7.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=5649', 'data', 'file=/tmp/tmpdgkgsmk6/uvb58qnc.json', 'init=/tmp/tmpdgkgsmk6/g24ljnu7.json', 'output', 'file=/tmp/tmpdgkgsmk6/prophet_modelzqcgz9n1/prophet_model-20250316201350.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
20:13:50 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
20:14:18 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done proc

{'MSE': [44.027273873991426], 'RMSE': [6.635305107829739]}
